##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Простое распознавание звука: распознавание ключевых слов

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/audio/simple_audio">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    Смотреть на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/audio/simple_audio.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Запустить в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/audio/simple_audio.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    Смотреть исходные файлы на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/audio/simple_audio.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачать ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

Из этого руководства Вы узнаете, как построить базовую сеть распознавания речи, которая распознает десять разных слов. Важно знать, что настоящие системы распознавания речи и звука намного сложнее, но, как и MNIST для изображений, они должны дать вам базовое понимание используемых методов. После того, как вы закончите это руководство, у вас будет модель, которая пытается классифицировать односекундный аудиоклип как «вниз», «вперед», «влево», «нет», «вправо», «стоп», «вверх» и "да".

## Установка

Для начала установим TensorFlow.

In [ ]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display


# Установите случайное начальное число(seed) для воспроизводимости эксперимента
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

## Импорт датасета голосовых команд

Скрипт начинается с загрузки части [набора данных речевых команд](https://www.tensorflow.org/datasets/catalog/speech_commands). Исходный набор данных состоит из более чем 105 000 аудиофайлов WAVE, в которых люди говорят тридцать разных слов. Эти данные были собраны Google и выпущены под лицензией CC BY, и вы можете помочь улучшить их, [предоставив пять минут своего собственного голоса](https://aiyprojects.withgoogle.com/open_speech_recording).

Вы будете использовать часть набора данных, чтобы сэкономить время на загрузке данных. Разархивируйте файл `mini_speech_commands.zip` и загрузите его с помощью `tf.data`.

In [ ]:
data_dir = pathlib.Path('data/mini_speech_commands')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'mini_speech_commands.zip',
      origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
      extract=True,
      cache_dir='.', cache_subdir='data')

Проверьте базовую статистику по датасету.

In [ ]:
commands = np.array(tf.io.gfile.listdir(str(data_dir)))
commands = commands[commands != 'README.md']
print('Commands:', commands)

Извлеките аудиофайлы в список и перемешайте список.

In [ ]:
filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)
print('Number of total examples:', num_samples)
print('Number of examples per label:',
      len(tf.io.gfile.listdir(str(data_dir/commands[0]))))
print('Example file tensor:', filenames[0])

Разделите файлы на датасеты для обучения, валидации и тестирования, используя соотношение 80:10:10 соответственно.

In [ ]:
train_files = filenames[:6400]
val_files = filenames[6400: 6400 + 800]
test_files = filenames[-800:]

print('Training set size', len(train_files))
print('Validation set size', len(val_files))
print('Test set size', len(test_files))

## Чтение аудиофайлов и их меток

Изначально аудиофайл будет считываться как двоичный файл, который вы преобразуете в числовой тензор.

Чтобы загрузить аудиофайл, вы будете использовать метод [`tf.audio.decode_wav`](https://www.tensorflow.org/api_docs/python/tf/audio/decode_wav). Этот метод принимает аудио в формате WAV и возвращает тензор и частоту дискретизации.

Файл WAV содержат данные таймсерий с заданным количеством выборок в секунду.
Каждая выборка представляет собой амплитуду аудиосигнала в определенное время. В 16-битной системе, как и в файлах датасета `mini_speech_commands`, значения варьируются от -32768 до 32767.
Частота дискретизации для этого набора данных составляет 16 кГц.
Обратите внимание, что `tf.audio.decode_wav` нормализует значения до диапазона [-1.0, 1.0].

In [ ]:
def decode_audio(audio_binary):
  audio, _ = tf.audio.decode_wav(audio_binary)
  return tf.squeeze(audio, axis=-1)

Меткой для файла wav является его родительский каталог.

In [ ]:
def get_label(file_path):
  parts = tf.strings.split(file_path, os.path.sep)
  # Примечание. Здесь вместо распаковки кортежей вы будете использовать индексирование, 
  # чтобы это работало в TensorFlow графе.
  return parts[-2] 

Давайте определим метод, который будет принимать имя файла wav и возвращать кортеж из аудио и меток.

In [ ]:
def get_waveform_and_label(file_path):
  label = get_label(file_path)
  audio_binary = tf.io.read_file(file_path)
  waveform = decode_audio(audio_binary)
  return waveform, label

Теперь примените `get_waveform_and_label` для создания обучающего набора с извлечением пар (аудио, метка) и проверки результатов. Позже вы создадите наборы для проверки и тестирования, используя аналогичную процедуру.

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
files_ds = tf.data.Dataset.from_tensor_slices(train_files)
waveform_ds = files_ds.map(get_waveform_and_label, num_parallel_calls=AUTOTUNE)

Давайте проверим несколько звуковых сигналов с соответствующими метками.

In [ ]:
rows = 3
cols = 3
n = rows*cols
fig, axes = plt.subplots(rows, cols, figsize=(10, 12))
for i, (audio, label) in enumerate(waveform_ds.take(n)):
  r = i // cols
  c = i % cols
  ax = axes[r][c]
  ax.plot(audio.numpy())
  ax.set_yticks(np.arange(-1.2, 1.2, 0.2))
  label = label.numpy().decode('utf-8')
  ax.set_title(label)

plt.show()

## Спектрограмма

Вы преобразуете сигнал в спектрограмму, которая показывает изменение частоты во времени и может быть представлена в виде двухмерного изображения. Это можно сделать, применив Оконное преобразование Фурье(STFT) для конвертирования звука в частотно-временное представление.

Преобразование Фурье ([`tf.signal.fft`](https://www.tensorflow.org/api_docs/python/tf/signal/fft)) преобразует сигнал в его составляющие частоты, но теряет всю информацию о времени. STFT ([`tf.signal.stft`](https://www.tensorflow.org/api_docs/python/tf/signal/stft)) разбивает сигнал на временные окна и выполняет преобразование Фурье в каждом окне, сохраняя некоторую информацию о времени и возвращая двумерный тензор, на котором вы можете запускать стандартные свертки.

STFT создает массив комплексных чисел, представляющих величину сигнала и фазу. Однако, в данном руководстве, вам понадобится только величина сигнала, которую можно получить, применив `tf.abs` к выходным данным `tf.signal.stft`.

Выберите параметры `frame_length`(размер окна) и `frame_step`(шаг окна) так, чтобы сгенерированное "изображение" спектрограммы было почти квадратным. Для получения дополнительной информации о выборе параметров STFT вы можете обратиться к [этому видео](https://www.coursera.org/lecture/audio-signal-processing/stft-2-tjEQe).

Вам также нужно, чтобы аудиограммы имели одинаковую длину, чтобы при преобразовании их в спектрограммы результаты имели аналогичные размеры. Это можно сделать путем простого заполнения нулями аудиоклипов, которые короче одной секунды. 

In [ ]:
def get_spectrogram(waveform):
  # Дополняем нулями файлы в которых менее 16000 сэмплов
  zero_padding = tf.zeros([16000] - tf.shape(waveform), dtype=tf.float32)

  # Объединяем аудио с дополнениями, чтобы все аудиоклипы были одинаковой длины
  waveform = tf.cast(waveform, tf.float32)
  equal_length = tf.concat([waveform, zero_padding], 0)
  spectrogram = tf.signal.stft(
      equal_length, frame_length=255, frame_step=128)
      
  spectrogram = tf.abs(spectrogram)

  return spectrogram

Изучите данные. Сравните форму волны, спектрограмму и фактическое аудио одного примера из набора данных.

In [ ]:
for waveform, label in waveform_ds.take(1):
  label = label.numpy().decode('utf-8')
  spectrogram = get_spectrogram(waveform)

print('Label:', label)
print('Waveform shape:', waveform.shape)
print('Spectrogram shape:', spectrogram.shape)
print('Audio playback')
display.display(display.Audio(waveform, rate=16000))

In [ ]:
def plot_spectrogram(spectrogram, ax):
  # Преобразовываем в частоты для логарифмической шкалы и транспонируем их так, 
  # чтобы время было представлено на оси x(столбцы).
  log_spec = np.log(spectrogram.T)
  height = log_spec.shape[0]
  X = np.arange(16000, step=height + 1)
  Y = range(height)
  ax.pcolormesh(X, Y, log_spec)


fig, axes = plt.subplots(2, figsize=(12, 8))
timescale = np.arange(waveform.shape[0])
axes[0].plot(timescale, waveform.numpy())
axes[0].set_title('Waveform')
axes[0].set_xlim([0, 16000])
plot_spectrogram(spectrogram.numpy(), axes[1])
axes[1].set_title('Spectrogram')
plt.show()

Теперь преобразуйте набор аудио-данных так, чтобы получить изображения спектрограммы и соответствующие им метки.

In [ ]:
def get_spectrogram_and_label_id(audio, label):
  spectrogram = get_spectrogram(audio)
  spectrogram = tf.expand_dims(spectrogram, -1)
  label_id = tf.argmax(label == commands)
  return spectrogram, label_id

In [ ]:
spectrogram_ds = waveform_ds.map(
    get_spectrogram_and_label_id, num_parallel_calls=AUTOTUNE)

Изучите «изображения» спектрограммы для различных примеров из набора данных.

In [ ]:
rows = 3
cols = 3
n = rows*cols
fig, axes = plt.subplots(rows, cols, figsize=(10, 10))
for i, (spectrogram, label_id) in enumerate(spectrogram_ds.take(n)):
  r = i // cols
  c = i % cols
  ax = axes[r][c]
  plot_spectrogram(np.squeeze(spectrogram.numpy()), ax)
  ax.set_title(commands[label_id.numpy()])
  ax.axis('off')
  
plt.show()

## Создание и обучение модели

Перед построением и обучением нашей модели вам нужно применить ту же обработку данных, что и на обучающем наборе, к проверочному и тестовому датасетам.

In [ ]:
def preprocess_dataset(files):
  files_ds = tf.data.Dataset.from_tensor_slices(files)
  output_ds = files_ds.map(get_waveform_and_label, num_parallel_calls=AUTOTUNE)
  output_ds = output_ds.map(
      get_spectrogram_and_label_id,  num_parallel_calls=AUTOTUNE)
  return output_ds

In [ ]:
train_ds = spectrogram_ds
val_ds = preprocess_dataset(val_files)
test_ds = preprocess_dataset(test_files)

Разделите на пакеты наборы для обучения и проверки.

In [ ]:
batch_size = 64
train_ds = train_ds.batch(batch_size)
val_ds = val_ds.batch(batch_size)

Добавьте методы датасета [`cache ()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#cache) и [`prefetch ()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#prefetch) для уменьшения задержки чтения при обучении модели.

In [ ]:
train_ds = train_ds.cache().prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)

В качестве модели вы будете использовать простую сверточную нейронную сеть(CNN), поскольку вы преобразовали аудиофайлы в изображения спектрограмм.
Модель также имеет следующие дополнительные слои предварительной обработки:
- Слой [`Resizing`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Resizing) для уменьшения размера входных данных, чтобы модель могла обучаться быстрее.
- Слой [`Normalization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Normalization) для нормализации каждого пикселя в изображении на основе его среднего значения и стандартного отклонения.

Для слоя `Normalization` сначала необходимо вызвать его метод `adapt`, чтобы вычислить совокупную статистику(т.е. среднее и стандартное отклонение).

In [ ]:
for spectrogram, _ in spectrogram_ds.take(1):
  input_shape = spectrogram.shape
print('Input shape:', input_shape)
num_labels = len(commands)

norm_layer = preprocessing.Normalization()
norm_layer.adapt(spectrogram_ds.map(lambda x, _: x))

model = models.Sequential([
    layers.Input(shape=input_shape),
    preprocessing.Resizing(32, 32), 
    norm_layer,
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_labels),
])

model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [ ]:
EPOCHS = 10
history = model.fit(
    train_ds, 
    validation_data=val_ds,  
    epochs=EPOCHS,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)

Давайте посмотрим кривые потерь при обучении и валидации, чтобы понять, насколько наша модель улучшилась за время обучения.

In [ ]:
metrics = history.history
plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

## Оценка эффективности на тестовом датасете

Запустим модель на тестовом наборе и проверим эффективность.

In [ ]:
test_audio = []
test_labels = []

for audio, label in test_ds:
  test_audio.append(audio.numpy())
  test_labels.append(label.numpy())

test_audio = np.array(test_audio)
test_labels = np.array(test_labels)

In [ ]:
y_pred = np.argmax(model.predict(test_audio), axis=1)
y_true = test_labels

test_acc = sum(y_pred == y_true) / len(y_true)
print(f'Test set accuracy: {test_acc:.0%}')

### Просмотр матрицы ошибок

Матрица ошибок полезна, чтобы увидеть, насколько хорошо модель справилась с каждой из команд в тестовом датасете.

In [ ]:
confusion_mtx = tf.math.confusion_matrix(y_true, y_pred) 
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, xticklabels=commands, yticklabels=commands, 
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()

## Выполнение вывода для аудиофайла

Наконец, проверьте результат предсказания модели, используя входной аудиофайл, в котором кто-то говорит «нет». Насколько хорошо работает наша модель?

In [ ]:
sample_file = data_dir/'no/01bb6a2a_nohash_0.wav'

sample_ds = preprocess_dataset([str(sample_file)])

for spectrogram, label in sample_ds.batch(1):
  prediction = model(spectrogram)
  plt.bar(commands, tf.nn.softmax(prediction[0]))
  plt.title(f'Predictions for "{commands[label[0]]}"')
  plt.show()

Вы можете видеть, что наша модель очень четко распознала звуковую команду как «нет».

## Следующие шаги

В этом руководстве показано, как можно выполнить простую классификацию аудио с помощью сверточной нейронной сети с использованием TensorFlow и Python.

* Чтобы узнать, как использовать переносное обучение для классификации аудио, ознакомьтесь с учебным пособием [Классификация звука с помощью YAMNet](https://www.tensorflow.org/hub/tutorials/yamnet).

* Чтобы создать свое собственное интерактивное веб-приложение для классификации аудио, подумайте о том, чтобы взять [TensorFlow.js - Распознавание аудио с использованием кода обучения с переносом](https://codelabs.developers.google.com/codelabs/tensorflowjs-audio-codelab/index.html#0).

* TensorFlow также имеет дополнительную поддержку для [подготовки и увеличения аудиоданных](https://www.tensorflow.org/io/tutorials/audio), чтобы помочь с вашими собственными аудио-проектами.